# Настройка окружения

Первый запуск:

`docker run --name goNotebook -it -p 8888:8888 -v /mnt/c/Users/Senya/Prog_2/:/notebooks gopherdata/gophernotes:latest-ds`

Последующий запуск контейнера

`docker start -i goNotebook`

Найти в выводе ссылку с токеном на нотебук и скопировать токен

http://127.0.0.1:8888/?token=здесьтокен

# Задачи

Ядро для `jupyter` - `gophernotes` ещё не доработано, 3 задачи не запустились в нотебуке, но прекрасно работают в отдельном файле  
Поэтому их решения дублируются отдельными файлами

In [33]:
import (
    "fmt"
    "time"
)

## 1 Встраивание
Дана структура Human (с произвольным набором полей и методов).  
Реализовать встраивание методов в структуре Action от родительской структуры  
Human (аналог наследования).


In [9]:
type Human struct {
    life bool
    hands int
}

func (h *Human) ShowHands() {
    fmt.Printf("Количество рук: %d", h.hands)
}

In [10]:
Nick := Human{false, 0}
Nick.ShowHands()

Количество рук: 0

In [16]:
type Action struct {
	Human
	handAmount int
}

func (a *Action) GiveHands()  {
	a.Human.hands = a.handAmount
    a.Human.ShowHands()
}

In [17]:
Nick_2 := Human{false, 0}
action := Action{Nick_2, 2}
action.GiveHands()

Количество рук: 2

## 2 Конкурентность
Написать программу, которая конкурентно рассчитает значение квадратов чисел  
взятых из массива (2,4,6,8,10) и выведет их квадраты в stdout.  

In [27]:
import "time"

arr := [...]int{2, 4, 6, 8, 10}


for id, val := range arr {
	go func (id, val int)  {
		arr[id] = val * val
	}(id, val)
}

time.Sleep(500 * time.Millisecond)

arr

[4 16 36 64 100]

## 3 Сумма квадратов
Дана последовательность чисел: 2,4,6,8,10. Найти сумму их  
квадратов(2²+3²+4²....) с использованием конкурентных вычислений.  

In [5]:
import "time"

arr := [...]int{2, 4, 6, 8, 10}
var sum int

for id, val := range arr {
	go func (id, val int)  {
		square := val * val
		arr[id] = square
		sum += square
	}(id, val)
}

time.Sleep(500 * time.Millisecond)

sum

220

## 4 Пул воркеров
Реализовать постоянную запись данных в канал (главный поток).  
Реализовать набор из N воркеров, которые читают произвольные данные из канала и
выводят в stdout.   
Необходима возможность выбора количества воркеров при
старте.

Программа должна завершаться по нажатию Ctrl+C. Выбрать и обосновать
способ завершения работы всех воркеров

Запустить и провериить можно в файле `4.go`

In [6]:
import (
	"fmt"
	"os"
	"os/signal"
	"syscall"
	"time"
)

In [ ]:
sigs := make(chan os.Signal)  // Канал с сигналами операционной системы
cancel := make(chan bool)     // Канал с уведомлением о отмене по Ctrl+C
main_thread := Writer(cancel) // Главный поток

const amount = 4

// Запуск воркеров
for i := 0; i < amount; i++ {
    go Worker(main_thread, PretendHelpfull)
}

// Канал оиждания прерывания
done := make(chan bool)
// Подписка на уведомление о прерывании
signal.Notify(sigs, syscall.SIGINT)

go func() {
    <-sigs         // Получение прерывания
    cancel <- true // Завершение горутин воркеров
    done <- true   // Завершение этой горутины
}()

<-done // Ожидание команды завершения

In [12]:
// Чтение канала
func Worker(inp chan int, work func(v int)) {
	for v := range inp {
		work(v)
	}
}

// Типа полезная работа
func PretendHelpfull(value int) {
	time.Sleep(500 * time.Millisecond)
	fmt.Print(value, ", ")
}

In [ ]:
// Запись в канал
func Writer(cancel chan bool) chan int {
	main_thread := make(chan int)

	go func() {
		i := 0
		defer close(main_thread)
		for {
			i++
			select {
			case main_thread <- i:
			case <-cancel:
				return
			}
		}
	}()

	return main_thread
}

## 5 Чтение канала

Разработать программу, которая будет последовательно отправлять  
значения в канал, а с другой стороны канала — читать.  
По истечению N секунд программа должна завершаться.


*(запуск в файле 5.go)*

In [ ]:
duration := 3

ch := Writer(duration)
go Reader(ch)

time.Sleep(time.Second * 4)

In [ ]:
func Writer(duration int) chan int {
	ch := make(chan int)
	count := 0

	alarm := time.After(time.Duration(duration * int(time.Second)))
	go func() {
		defer close(ch)
		for {
			count++
			time.Sleep(time.Millisecond * 250)
			select {
			case ch <- count:
			case t := <-alarm:
				return
			}
		}
	}()

	return ch
}

In [21]:
func Reader(inp chan int)  {
	for val := range inp {
		fmt.Printf("Считал - %d", val)
	}
}

## 6 Остановка горутин
Реализовать все возможные способы остановки выполнения горутины.

Закрытие канала


In [ ]:
ch := make(chan bool)

go func() {
	for val := range ch {
		fmt.Println(val)
	}
	fmt.Println("Завершение")
}()

ch<-false
ch<-false

close(ch)


wg.Done


In [ ]:
import "sync"

var wg sync.WaitGroup

wg.Add(1)

go func(wg sync.WaitGroup) {
	fmt.Println("hello")
	wg.Done()
}(wg)

wg.Wait()


runtime.Goexit()

In [ ]:
import "runtime"

func exit() {
    fmt.Println("Старт горутины")
    defer fmt.Println("Отложеннный вызов в горутине")
    runtime.Goexit()
}

go exit()

time.Sleep(time.Second * 2)

## 7 Конкурентная запись в map
Реализовать конкурентную запись данных в map.

In [5]:
import (
	"fmt"
	"sync"
)

m := map[int]int{}
mux := sync.Mutex{}

writer := func (m map[int]int, mux *sync.Mutex, wg *sync.WaitGroup)  {
	idx := 1				// Если не лочить - произойдёт гонка
	mux.Lock()				// гонка - одновременная запись по одному ключу.  
	m[idx]++				// не засчитается одна из операций, нарушится целостность данных
	mux.Unlock()
	wg.Done()
}


var wg sync.WaitGroup

for i := 0; i < 100; i++ {
	wg.Add(1)
	go writer(m, &mux, &wg)
}

wg.Wait()
fmt.Printf("%#v", m)

map[int]int{1:100}

18 <nil>

## 8 Установка бита
Дана переменная int64. Разработать программу которая устанавливает i-й бит в 1 или 0.


In [34]:
import "math"

In [184]:
func changeBit(num int64, bit uint) {
	mask := int64(1 << bit)

	fmt.Println("   XOR")
	fmt.Printf("%08b - маска\n    ↓\n", mask)

	num = num ^ mask

	fmt.Printf("%08b - результат\n", num)
}

In [185]:
var num int64 = 42
fmt.Printf("%08b - исходное число\n", num)	// Посмотреть битовое представление
changeBit(num, 5)							// Изменить iый бит

00101010 - исходное число
   XOR
00100000 - маска
    ↓
00001010 - результат


## 9 Конвейер чисел
Разработать конвейер чисел. Даны два канала: в первый пишутся числа (x) из массива, во второй — результат операции x*2, после чего данные из второго канала должны выводиться в stdout.


In [9]:
arr := []int{3, 5, 7, 9, 11, 13}

nums := writer(arr)				// Запись чисел из массива в канал
power_nums := givePower(nums)	// Запись x^2 в канал
showPower(power_nums)			// Вывод

9
25
49
81
121
169


In [ ]:
func writer(arr []int) chan int {
	out := make(chan int)

	go func(arr []int) {
		defer close(out)		// Почему то не работает в нотебуке
		for _, v := range arr {
			out <- v
		}
	}(arr)

	return out
}

In [ ]:
func givePower(inp <-chan int) chan int {
	out := make(chan int)
	go func() {
		defer close(out)
		for v := range inp {
			out <- (v * v)
		}
	}()
	return out
}

In [4]:
import "fmt"

func showPower(inp chan int) {
	for v := range inp {
		fmt.Println(v)
	}
}

## 10 Распределение температур
Дана последовательность температурных колебаний: -25.4, -27.0 13.0, 19.0,  
15.5, 24.5, -21.0, 32.5. Объединить данные значения в группы с шагом в 10 градусов.  
Последовательность в подмножноствах не важна.  
Пример: -20:{-25.0, -27.0, -21.0}, 10:{13.0, 19.0, 15.5}, 20: {24.5}, etc.


In [41]:
temps := []float32{-25.4, -27.0, 13.0, 19.0, 15.5, 24.5, -21.0, 32.5}
temp_group := map[int][]float32{}

for _, v := range temps {
	group := int(v) / 10 * 10
	temp_group[group] = append(temp_group[group], v)
}
temp_group

map[10:[13 19 15.5] 20:[24.5] 30:[32.5] -20:[-25.4 -27 -21]]

## 11 Пересечение множеств
Реализовать пересечение двух неупорядоченных множеств.


In [60]:
set_1 := []int{2, 4, 5, 7}
set_2 := []int{1, 3, 5, 7}

matched := intersection(set_1, set_2)
matched

[5 7]

In [58]:
func intersection(s1, s2 []int) []int {
	matched := make([]int, 0)

	for _, v1 := range s1 {
		for _, v2 := range s2 {
			if v1 == v2 {
				matched = append(matched, v1)
			}
		}
	}
	return matched
}

## 12 Множество строк
Имеется последовательность строк - (cat, cat, dog, cat, tree) создать для нее
собственное множество.

In [52]:
seq := []string{"cat", "cat", "dog", "cat", "tree"}

set := makeSet(seq)
set

[cat dog tree]

In [51]:
func makeSet(seq []string) []string {
	set := make([]string, 0)

	contains := func(s []string, ref string) bool {
		for _, compared := range s {
			if compared == ref {
				return true
			}
		}
		return false
	}

	for _, v := range seq {
		if !contains(set, v) {
			set = append(set, v)
		}
	}
	return set
}

## 13 Распаковка переменных
Поменять местами два числа без создания временной переменной.


In [14]:
a, b := 1, 3
a, b = b, a

fmt.Println(a, b); ""

3 1


## 14 Определение типа
Разработать программу, которая в рантайме способна определить тип
переменной: int, string, bool, channel из переменной типа interface{}.


In [17]:
func do(i interface{}) {
	switch v := i.(type) {
	case int:
		fmt.Printf("Twice %v is %v\n", v, v*2)
	case string:
		fmt.Printf("%q is %v bytes long\n", v, len(v))
	default:
		fmt.Printf("I don't know about type %T!\n", v)
	}
}

In [21]:
do(13)
do("ffffuck")
do(false)

Twice 13 is 26
"ffffuck" is 7 bytes long
I don't know about type bool!


## 15 Утечка памяти
К каким негативным последствиям может привести данный фрагмент кода, и как
это исправить?  
Приведите корректный пример реализации.

```go
var justString string
func someFunc() {
    v := createHugeString(1 << 10)
    justString = v[:100]
}

func main() {
    someFunc()
}
```

Глобальная `justString` ссылается на первые 100 байт строки `v`,  
из за этого в куче требуется целиком хранить строку `v`,  
`v` не используется нигде кроме фукции - это приводит к утечке памяти

Корректный пример:
```go
justString := make([]byte, 100)

func someFunc() {
    v := createHugeString(1 << 10)
    copy(justString, v[:100])       // Теперь на v не будет лишних ссылок и сборщик мусора её удалит
}

func main() {
    someFunc()
}
```

## 16 Быстрая сортировка
Реализовать быструю сортировку массива (quicksort) встроенными методами
языка.

In [2]:
import (
	"fmt"
	"sort"
	"math/rand"
)

func genNums(len int) []int {
	arr := make([]int, len)
	for k, _ := range arr {
		arr[k] = rand.Intn(10)
	}
	return arr
}

In [3]:
func Split(n []int, l, r int) int {
	pole := l
	for {
		fmt.Println(n, l, r)
		for l < len(n) && n[l] < n[pole] {
			l++
		}
		for r >= 0 && n[r] > n[pole] {
			r--
		}
		if l >= r {
			return r+1
		}
		n[l], n[r] = n[r], n[l]
		l++; r--
	}
}

func sort(n []int, l, r int)  {
	if l < r {
		idx := Split(n, l, r)
		fmt.Println(n[l:idx], n[idx:r+1])
		sort(n, l, idx-1)
		sort(n, idx, r)
	}
}

In [4]:
arr := genNums(6)
fmt.Println(arr)

sort(arr, 0, len(arr)-1)
arr

[1 7 7 9 1 8]
[1 7 7 9 1 8] 0 5
[1 7 7 9 1 8] 1 3
[1] [7 7 9 1 8]
[1 7 7 9 1 8] 1 5
[1 1 7 9 7 8] 2 3
[1] [7 9 7 8]
[1 1 7 9 7 8] 2 5
[1 1 7 9 7 8] 3 3
[7] [9 7 8]
[1 1 7 9 7 8] 3 5
[1 1 7 8 7 9] 4 4
[8 7] [9]
[1 1 7 8 7 9] 3 4
[1 1 7 7 8 9] 4 3
[7] [8]


[1 1 7 7 8 9]

## 17 Бинарный поиск
Реализовать бинарный поиск встроенными методами языка.


In [1]:
import (
	"fmt"
	"sort"
	"math/rand"
)

func genNums(len int) []int {
	arr := make([]int, len)
	for k, _ := range arr {
		arr[k] = rand.Intn(100)
	}
	return arr
}

func Sort(n []int) {
	sort.Slice(n, func (i, j int) bool {
		return n[i] < n[j]
	})
}

In [67]:
func binSearch(arr []int, v int) int {
	l := len(arr)
	left, right := 0, l
	for {
		l = right - left
		split := l / 2 + left
		fmt.Println(arr[left:split], " | ", arr[split], " | ", arr[split+1:right])

		if arr[split] == v {
			return split
		}

		if v < arr[split] {
			right = split
		} else if v > arr[split] {
			left = split+1
		}

		if right - left == l {
			return -1
		}
	}
}

In [71]:
arr := genNums(33)
Sort(arr)
arr

[1 2 4 5 8 10 15 22 23 31 35 39 40 40 40 42 43 44 51 51 54 57 58 62 64 66 67 68 76 78 83 86 90]

In [72]:
const val = 54
binSearch(arr, val)

[1 2 4 5 8 10 15 22 23 31 35 39 40 40 40 42]  |  43  |  [44 51 51 54 57 58 62 64 66 67 68 76 78 83 86 90]
[44 51 51 54 57 58 62 64]  |  66  |  [67 68 76 78 83 86 90]
[44 51 51 54]  |  57  |  [58 62 64]
[44 51]  |  51  |  [54]
[]  |  54  |  []


20

## 18 Конкурентная структура инкремент
Реализовать структуру-счетчик, которая будет инкрементироваться в
конкурентной среде. По завершению программа должна выводить итоговое
значение счетчика.

In [13]:
import "sync/atomic"

type counter struct {
	amount int32
}

func (c *counter) incr()  {
	atomic.AddInt32(&c.amount, 1)	
}

In [46]:
count := counter{}

for i := 0; i < 100; i++ {
	go func() {
		count.incr()
	}()
}
time.Sleep(time.Second / 100)
count

{100}

## 19 Разворот unicode строки
Разработать программу, которая переворачивает подаваемую на ход строку
(например: «главрыба — абырвалг»).  
Символы могут быть unicode.


In [30]:
import "unicode/utf8"

func reverse(s string) string {
	r := []rune(s)
	n := utf8.RuneCountInString(s)

	for i, j := 0, n-1; i < j; i, j = i+1, j-1 {
		r[i], r[j] = r[j], r[i]
	}

	return string(r)
}

In [31]:
reverse("главрыбаb")

bабырвалг

## 20 Разворот предложения
Разработать программу, которая переворачивает слова в строке.
Пример: «snow dog sun — sun dog snow».


In [41]:
import "strings"

func reverseSentence(s string) []string {
	words := strings.Fields(s)
	res := make([]string, len(words))
	for i, j := 0, len(words)-1; i < j; i, j = i+1, j-1 {
		res[i], res[j] = words[j], words[i]
	}
	return res
}

In [43]:
reverseSentence("привет всем привет мир")

[мир привет всем привет]

## 21 Адаптер
Реализовать паттерн «адаптер» на любом примере.



Существующий интерфейс

In [55]:
type PixelMap interface {
	setPix(x, y int)
}

// Структура с координатами
type pixelMap struct {
	width, height int
	coords [][]int
}

func (pm *pixelMap) setPix(x, y int)  {
	// ... Проверки, преобразования, суета
	pm.coords[x][y] = x^y
}

Интерфейс с специфичным методом  
Требуется использовать уже существующий pixelMap

In [57]:
type Pencil interface {
	drawCircle(x, y, r int)
}

// Структура с координатами в абстрактной системе координат
type pencil struct {
	abstrctDots [][]int		// Двумерный срез координат 
}

func (p *pencil) drawCircle(x, y, r int)  {
	for i := 0; i < 360; i++ {
		dot := make([]int, 2)
		// ... Вычисление координат
		p.abstrctDots = append(p.abstrctDots, dot)
	}
}

Адаптер - реализует специфичный интерфейс методами существующего

In [61]:
type pencilPixAdapter struct {
	pencil
	pixelMap
}

func (pa *pencilPixAdapter) drawCircle(x, y, r int)  {
	for _, coord := range pa.pencil.abstrctDots {
		// Преобразование из одной системы координат в другую
		pa.pixelMap.setPix(coord[0], coord[1])
	}
}

Использование

In [68]:
pencl := pencil{[][]int{} }
pixMap := pixelMap{16, 9, [][]int{}}
adapter := pencilPixAdapter{pencl, pixMap}

## 22 Арифметика больших int'ов
Разработать программу, которая перемножает, делит, складывает, вычитает  
две числовых переменных a,b, значение которых > 2^20.

In [88]:
import "math/big"

a := big.NewInt(2e7)
b := big.NewInt(3e7)
fmt.Println(a, b)

// умножение
result := new(big.Int).Mul(a, b)
fmt.Println(result)

// деление
result = new(big.Int).Div(b, a)
fmt.Println(result)

// сложение
result = new(big.Int).Add(a, b)
fmt.Println(result)

// вычитание
result = new(big.Int).Sub(a, b)
fmt.Println(result)

20000000 30000000
600000000000000
1
50000000
-10000000


10 <nil>

## 23 Удалить i-ый элемент из слайса.


In [69]:
// Если порядок в срезе не важен
func remove(s []int, i int) []int {
    s[i] = s[len(s)-1]
    return s[:len(s)-1]
}

In [63]:
// Если порядок важен и некуда спешить
func remove(slice []int, s int) []int {
    return append(slice[:s], slice[s+1:]...)
}

In [77]:
slc := make([]int, 10)
slc[4], slc[9] = 5, 10
fmt.Println(slc)
remove(slc, 4)

[0 0 0 0 5 0 0 0 0 10]


[0 0 0 0 10 0 0 0 0]

## 24 Инкапсулированное расстояние
Разработать программу нахождения расстояния между двумя точками, которые  
представлены в виде структуры Point с инкапсулированными параметрами x,y и конструктором.

In [ ]:
import "math"

// Структура поля которой необходимо инкапсулировать
type point struct {
	x, y float64	// Недоступны из внешних пакетов так как с маленькой буквы
}

// Получение координат
func (p point) Coords() (float64, float64) {
	return p.x, p.y
}

// Метод нахождения расстояния
func (p point) Distance(p2 PointFace) float64 {
	x2, y2 := p2.Coords()
	dx := p.x - x2
	dy := p.y - y2
	return math.Sqrt(dx*dx - dy*dy)
}


In [29]:
// Интерфейс для инкапсуляции
type PointFace interface {
	Coords() (float64, float64)
	Distance(p2 PointFace) float64
}

// Встраивание интерфейса
type Point struct {
	PointFace
}

// Констрктор
func NewPoint(x, y float64) Point {
	return Point{point{x: x, y: y}}
}

In [19]:
p := NewPoint(5, 5)
q := NewPoint(10, 10)
fmt.Printf("Тип: %#v\n\n", p)

d := q.Distance(p)
fmt.Println(d)

// p.x -> p.x undefined (type Point has no field or method x)

Здесь код не работает, поэтому решение продублировано в файле `24.go`  
Вывод программы:
```
Тип: main.Point{PointFace:main.point{x:5, y:5}}

7.0710678118654755
```

## 25 Реализовать собственную функцию sleep.


In [34]:
func sleep(n int) {
	start := time.Now()
	// Пока: текущее время - время старта < длительность сна
	for time.Now().Sub(start) < time.Duration(n*int(time.Second)) {
	}
}


In [36]:
// Тоже самое но обнаружил существование Since
func sleep(n int) {
	start := time.Now()
	// Пока: время прошедшее со старта < длительность сна
	for time.Since(start) < time.Duration(n*int(time.Second)) {
	}
}


In [37]:
start := time.Now()
sleep(5)
end := time.Since(start)
fmt.Println("Took: ", end)

Took:  5.0000077s


18 <nil>

## 26 Уникальные символы слова
Разработать программу, которая проверяет, что все символы в строке  
уникальные (true — если уникальные, false etc). Функция проверки должна быть регистронезависимой.

Например:
```
abcd — true
abCdefAaf — false
aabcd — fals
```

In [41]:
import "strings"


func uniq(s string) bool {
	s = strings.ToLower(s)
	charSet := make([]rune, len(s))
	duplicates := func (r rune) bool {
		for _, char := range charSet {
			if r == char {
				return true
			}
		}
		return false
	}

	for _, char := range []rune(s) {
		if duplicates(char) {
			return false
		}
		charSet = append(charSet, char)
	}
	return true
}

In [51]:
uniq("abvгд")

true

In [50]:
uniq("абв Gg")

false

# Вопросы 

### 1 Какой самый эффективный способ конкатенации строк?

*Паподробней...*

`strings.Builder` - избегает дополнительных копирований

### 2 Что такое интерфейсы, как они применяются в Go?

Интерфейс в го - набор сигнатур методов которые реализует структура

Сигнатура - описание функции - тип принимаемых аргументов, тип возвращаемых значений

Структура удовлетворяет интерфейсу, если реализует все методы интерфейса

Интерфейсы применяются для абстрагирования от реализации,  
структура имеющая интерфейс может иметь разные реализации методов

Так же с помощью интерфейсов можно скрывать поля структур

```go
type face struct {
	eye int
}

func (f *face) Blink()  {
	if f.eye >= 2 {
		fmt.Println("😉")
	}
}

type Action interface {
	Blink()
}

type Head struct {
	Action
}

h := Head{&face{2}}
h.Blink()
>>> 😉
h.eye
>>> type main.Head has no field or method "eye": h.eye
```

### 3 Чем отличаются RWMutex от Mutex?

Mutex - инструмент гарантирующий, что определённые общие данные или участок кода,  
изменяет/выполняет только одна горутина. Защита от одновременного доступа

RWMutex - позволяет блокировать участок кода только для записи.

Горутины читатели, если используют `.RLock()`, имеют одновременный доступ к участку кода не блокируя его друг для друга  
Горутина писатель, заблокировавшая `.Lock()` заблокирует участок для всех 

### 4 Чем отличаются буферизированные и не буферизированные каналы?
Запись в буферизированный канал не блокирует выполнение, кода пока буффер не будет заполнен

Запись и чтение осуществляется через очередь:  
Первый записанный будет первым считан

После закрытия канала, сначала считаываются все находящиеся в буффере значения, после этого считывание возвращает nil

### 5 Какой размер у структуры struct{}{}?
0 байт 

### 6 Есть ли в Go перегрузка методов или операторов?
Неа  
Таков путь  

Для передачи одного опционального параметра, можно использовать синтаксис множественых аргументов

```go
func test(arg1 int, optional ...int)  {
	fmt.Println(arg1, optional)
}

test(1); test(1, 2)

>> 1 []
>> 1 [2]
```

### 7 В какой последовательности будут выведены элементы map[int]int?
Пример:

m[0]=1  
m[1]=124  
m[2]=281  

В случайном порядке.

### 8. В чем разница make и new?

make - создание одного из нескольких встроенных типов данных: `slice`, `map` или `channel`

new - создание указателя на новый объект



### 9 Сколько существует способов задать переменную типа slice или map?

Литералы (без выделения памяти):
```go
a := []int{1, 2, 3} // slice
m := map[string]int{"a": 1, "b": 2, "c": 3} // map
```
Создание пустого slice или map:
```go
a := make([]int, 0)
m := make(map[string]int)
```
Инициализация с заранее выделенной памятью:
```go
a := make([]int, 3, 5) // slice с длиной 3 и емкостью 5
m := make(map[string]int, 100) // map с емкостью 100
```
Без инициализации и присвоения значений:
```go
var a []int
var m map[string]int
```

### 10 Что выведет данная программа и почему?

```go
func update(p *int) {
    b := 2
    p = &b
}

func main() {
    var (
        a = 1
        p = &a
    )
    fmt.Println(*p)
    update(p)
    fmt.Println(*p)
}
```

Выведет 1 и 2  
Потому что по ссылке изменяется значение переменной

### 11 Что выведет данная программа и почему?
```go
func main() {
    wg := sync.WaitGroup{}
    for i := 0; i < 5; i++ {
        wg.Add(1)
        go func(wg sync.WaitGroup, i int) {
            fmt.Println(i)
            wg.Done()
        }(wg, i)
    }
    wg.Wait()
    fmt.Println("exit")
}
```

Программа выведет принты и попадёт в вечное ожидание,  
группа ожидания передавалась в горутины по значению - каждый раз создовалась новая  

Что бы этого избежать, следует передавать группу по ссылке, в данном примере можно обойтись без передачи из за замыкания

In [2]:
import (
	"fmt"
	"sync"
)
wg := sync.WaitGroup{}
for i := 0; i < 5; i++ {
	wg.Add(1)
	go func(i int) {
		fmt.Println(i)
		wg.Done()
	}(i)
}
wg.Wait()
fmt.Println("exit")

1
3
4
2
0
exit


5 <nil>

### 12 Что выведет данная программа и почему?
```go
func main() {
    n := 0
    if true{
        n := 1
        n++
    }
    fmt.Println(n)
}
```

Программа выведет 0  
Внутри if создаётся и инкрементируется новая - локальная n  

При выходе из if она стирается из памяти и выводится ранее созданная n со значением 0

### 13 Что выведет данная программа и почему?
```go
func someAction(v []int8, b int8) {
    v[0] = 100
    v = append(v, b)
}

func main() {
    var a = []int8{1, 2, 3, 4, 5}
    someAction(a, 6)
    fmt.Println(a)
}
```
Программа выведет [100 2 3 4 5]  
Срезы являются ссылкой на массив  
При передаче срезов в функции, значения не копируются, изменятся исходный массив

Для массивов всё наоборот - массивы являются значением  
При передаче в функцию полностью копируются

In [60]:
a := [...]int{0: 1, 3: 3}

func y(a [4]int)  {
	a[0] = 0
}

fmt.Println(a)
y(a)
a

[1 0 0 3]


[1 0 0 3]

### 14 Что выведет данная программа и почему?
```go
func main() {
    slice := []string{"a", "a"}
    
    func(slice []string) {
        slice = append(slice, "a")
        slice[0] = "b"
        slice[1] = "b"
        fmt.Print(slice)
    }(slice)

    fmt.Print(slice)
}
```

Программа выведет `[b b a][a a]`

Возвращаемое значение `append` - новый срез с добавленными элементами   
Поэтому в анонимной функции изменяется не исходный, а новый срез   


In [73]:
slice := []string{"a", "a"}

// Можно сделать грязи с указателями
a := func(slice *[]string) {
	*slice = append(*slice, "a")
	(*slice)[0] = "b"
	(*slice)[1] = "b"
	fmt.Print(*slice)
}
a(&slice)

fmt.Print(slice)

[b b a][b b a]

7 <nil>

In [75]:
slice := []string{"a", "a"}

// А лучше просто вернуть новый срез
a := func(slice []string) []string {
	slice = append(slice, "a")
	slice[0] = "b"
	slice[1] = "b"
	fmt.Print(slice)
	return slice
}
slice = a(slice)

fmt.Print(slice)

[b b a][b b a]

7 <nil>